In [1]:
"""
Notebook Matrix Profile – ampiimts Python 3.11 package

Goal:
From pre processing signal with original value, normalize value, timestamp. 
We identify discord and motifs with a fix window on matrix profile methode (stumpy.maamp)
"""
import numpy as np
import pandas as pd
import ampiimts
import time
import os
import stumpy

In [2]:
#import files
folder = '../../dataset/pollution'
pds = []
with os.scandir(folder) as entries:
    for entry in entries:
        if entry.is_file() and entry.name.endswith('csv'):
            # list of sensors with some parameters -> list[pd.DataFrame[timestamp, value1, value2, etc..., latitude, longitude]]) 
            pds.append(pd.read_csv(folder+'/'+entry.name))
len(pds)
pds = pds[:10]
# and not entry.name.startswith('pollutionData')

In [ ]:
pds_normalized = ampiimts.pre_processed(pds, sort_by_variables=False)

mx_profile = ampiimts.matrix_profile(pds_normalized)



[WINDOW LIST] Fenêtre retenue (variables séparées) → 14 days 02:40:00


KeyboardInterrupt: 

In [ ]:

print(mx_profile[0])

# ampiimts.plot_multidim_matrix_profile(pds_normalized[0], mx_profile[0])

{'patterns': [], 'matrix_profile':                     value
2014-08-01 00:05:00   NaN
2014-08-01 00:10:00   NaN
2014-08-01 00:15:00   NaN
2014-08-01 00:20:00   NaN
2014-08-01 00:25:00   NaN
...                   ...
2014-09-30 23:40:00   NaN
2014-09-30 23:45:00   NaN
2014-09-30 23:50:00   NaN
2014-09-30 23:55:00   NaN
2014-10-01 00:00:00   NaN

[17568 rows x 1 columns], 'discord_indices': array([2715, 2716, 2717, 2718, 2719, 2720, 2721, 2722, 2723, 2724, 2725,
       2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733, 2734, 2735, 2736,
       2737, 2738, 2739, 2740, 2741, 2742, 2743, 2744, 2745, 2746, 2747,
       2748, 2749, 2750, 2751, 2752, 2753, 2754, 2755, 2756, 2757, 2758,
       2759, 2760, 2761, 2762, 2763, 2764, 2765, 2766, 2767, 2768, 2769,
       2770, 2771, 2772, 2773, 2774, 2775, 2776, 2777, 2778, 2779, 2780,
       2781, 2782, 2783, 2784, 2785, 2786, 2787, 2788, 2789, 2790, 2791,
       2792, 2793, 2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802,
       2803, 2804, 2805